In [94]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent))

In [95]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

In [96]:
def f(x: np.ndarray) -> float:
    return x[0] ** 2 + x[1] ** 2 + 4 * x[1] - 1

In [97]:
def g1(x: np.ndarray) -> float:
    return x[0] ** 2 + x[1]


def g2(x: np.ndarray) -> float:
    return x[0] - 2 * x[1] - 8

In [98]:
def opt_penalty_function(x: np.ndarray, mu: float) -> np.ndarray:
    def foo(y: np.ndarray) -> float:
        
        return f(y) + mu * (
            (max(0, g1(y))) ** 2 + 
            (max(0, g2(y))) ** 2
        )
    
    opt = minimize(foo, x).x
    
    return opt, foo(opt)

In [99]:
def opt_barrier_function(x: np.ndarray, mu: float) -> np.ndarray:
    def foo(y: np.ndarray) -> float:
        
        return f(y) - mu * (
            (1 / g1(y)) + (1 / g2(y))
        )
    
    opt = minimize(foo, x, constraints=(
        NonlinearConstraint(g1, -np.inf, -1e-7),
        NonlinearConstraint(g2, -np.inf, -1e-7)
                                        )).x
    
    return opt, foo(opt)

In [100]:
START = np.array([0.5, -1.5])
EPS = 0.05
MU_1 = 0.001
BETA_1 = 10
MU_2 = 1
BETA_2 = 0.4

In [101]:
def penalty_function_method(x: np.ndarray, eps: float, mu: float, beta: float) -> np.ndarray:
    x_1 = x
    mu_1 = mu

    while True:
        x_1, barrier = opt_barrier_function(x_1, mu_1)

        if mu_1 * barrier < eps:
            return x_1
        
        mu_1 *= beta


print(penalty_function_method(START, EPS, MU_1, BETA_1))        

[-1.10074961e-04 -2.00003605e+00]


In [102]:
def barrier_function_method(x: np.ndarray, eps: float, mu: float, beta: float) -> np.ndarray:
    x_1 = x
    mu_1 = mu

    while True:
        x_1, penalty = opt_penalty_function(x_1, mu_1)

        if mu_1 * penalty < eps:
            return x_1
        
        mu_1 *= beta


print(barrier_function_method(START, EPS, MU_2, BETA_2))

[-1.46311152e-09 -2.00000000e+00]
